# Importance des poids sur les couches du modèle Vgg16

In [1]:
from tensorflow.keras.applications import VGG16

# Charger le modèle pré-entraîné (VGG16)
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in vgg16.layers:
    layer.trainable = False

vgg16.summary()

58889256/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                              

# Test Grad-CAM sur images avec background

In [ ]:
import pickle
import tensorflow as tf
import cv2
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt

response = requests.get('https://drive.google.com/uc?export=view&id=1MX2Zz7Q9qGejgkmK5VfwBTkabR2-6soN')
img1 = Image.open(BytesIO(response.content))

input_shape = (256, 256, 3)

# Chargement du modèle
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Modèles de gradient pour chaque couche
grad_models = [
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_pool').output, model.output])
]

# Prétraitement de l'image
img = img1.resize((256, 256))
img = np.array(img) / 255.0
img = np.expand_dims(img, axis=0)

num_rows = 5
num_cols = 4
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))
fig.suptitle('Grad-CAM Visualization', fontsize=16)

# Image originale
axs[0, 0].imshow(img1)
axs[0, 0].set_title('Original Image')
axs[0, 0].axis('off')

for i, grad_model in enumerate(grad_models):
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img)
        label_idx = tf.argmax(tf.squeeze(predictions)).numpy()  # Index de la classe prédite
        loss = tf.gather(tf.squeeze(predictions), [label_idx])

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]
    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = gate_f * gate_r * grads
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, output), axis=-1)
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (256, 256))
    heatmap = cam / np.max(cam)
    overlay = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    gradcam = cv2.addWeighted(np.array(img1), 0.5, overlay, 0.5, 0)

    row = (i + 1) // num_cols
    col = (i + 1) % num_cols

    axs[row, col].imshow(gradcam, cmap='jet')
    axs[row, col].set_title(f'block{i+1}_layer')
    axs[row, col].axis('off')

# Suppression des subplots vides
for i in range(len(grad_models) + 1, num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Apple___healthy

In [ ]:
import pickle
import tensorflow as tf
import cv2
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt

response = requests.get('https://drive.google.com/uc?export=view&id=17oggth06ZSBv1v5I4_XGx64gEqyBenf2')
img1 = Image.open(BytesIO(response.content))

input_shape = (256, 256, 3)

# Chargement du modèle
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Modèles de gradient pour chaque couche
grad_models = [
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_pool').output, model.output])
]

# Prétraitement de l'image
img = img1.resize((256, 256))
img = np.array(img) / 255.0
img = np.expand_dims(img, axis=0)

num_rows = 5
num_cols = 4
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))
fig.suptitle('Grad-CAM Visualization', fontsize=16)

# Image originale
axs[0, 0].imshow(img1)
axs[0, 0].set_title('Original Image')
axs[0, 0].axis('off')

for i, grad_model in enumerate(grad_models):
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img)
        label_idx = tf.argmax(tf.squeeze(predictions)).numpy()  # Index de la classe prédite
        loss = tf.gather(tf.squeeze(predictions), [label_idx])

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]
    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = gate_f * gate_r * grads
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, output), axis=-1)
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (256, 256))
    heatmap = cam / np.max(cam)
    overlay = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    gradcam = cv2.addWeighted(np.array(img1), 0.5, overlay, 0.5, 0)

    row = (i + 1) // num_cols
    col = (i + 1) % num_cols

    axs[row, col].imshow(gradcam, cmap='jet')
    axs[row, col].set_title(f'block{i+1}_layer')
    axs[row, col].axis('off')

# Suppression des subplots vides
for i in range(len(grad_models) + 1, num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Apple___Apple_scab

In [ ]:
import pickle
import tensorflow as tf
import cv2
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt

response = requests.get('https://drive.google.com/uc?export=download&id=19c6jMfb-7iX1kWb9HllLxkpN4oI2Mz0s')
img1 = Image.open(BytesIO(response.content))

input_shape = (256, 256, 3)

# Chargement du modèle
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Modèles de gradient pour chaque couche
grad_models = [
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_pool').output, model.output])
]

# Prétraitement de l'image
img = img1.resize((256, 256))
img = np.array(img) / 255.0
img = np.expand_dims(img, axis=0)

num_rows = 5
num_cols = 4
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))

# Image originale
axs[0, 0].imshow(img1)
axs[0, 0].set_title('Image originale')
axs[0, 0].axis('off')

for i, grad_model in enumerate(grad_models):
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img)
        label_idx = tf.argmax(tf.squeeze(predictions)).numpy()  # Index de la classe prédite
        loss = tf.gather(tf.squeeze(predictions), [label_idx])

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]
    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = gate_f * gate_r * grads
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, output), axis=-1)
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (256, 256))
    heatmap = cam / np.max(cam)
    overlay = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    gradcam = cv2.addWeighted(np.array(img1), 0.5, overlay, 0.5, 0)

    row = (i + 1) // num_cols
    col = (i + 1) % num_cols

    axs[row, col].imshow(gradcam, cmap='jet')
    axs[row, col].set_title(f'block{i+1}_layer')
    axs[row, col].axis('off')

# Suppression des subplots vides
for i in range(len(grad_models) + 1, num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Grape___Black_rot

In [ ]:
import pickle
import tensorflow as tf
import cv2
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt

response = requests.get('https://drive.google.com/uc?export=download&id=1cL1gFMQEI7jRIJXEcZ4d8k0zIo74xkPq')
img1 = Image.open(BytesIO(response.content))

input_shape = (256, 256, 3)

# Chargement du modèle
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Modèles de gradient pour chaque couche
grad_models = [
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_pool').output, model.output])
]

# Prétraitement de l'image
img = img1.resize((256, 256))
img = np.array(img) / 255.0
img = np.expand_dims(img, axis=0)

num_rows = 5
num_cols = 4
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))

# Image originale
axs[0, 0].imshow(img1)
axs[0, 0].set_title('Image originale')
axs[0, 0].axis('off')

for i, grad_model in enumerate(grad_models):
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img)
        label_idx = tf.argmax(tf.squeeze(predictions)).numpy()  # Index de la classe prédite
        loss = tf.gather(tf.squeeze(predictions), [label_idx])

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]
    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = gate_f * gate_r * grads
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, output), axis=-1)
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (256, 256))
    heatmap = cam / np.max(cam)
    overlay = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    gradcam = cv2.addWeighted(np.array(img1), 0.5, overlay, 0.5, 0)

    row = (i + 1) // num_cols
    col = (i + 1) % num_cols

    axs[row, col].imshow(gradcam, cmap='jet')
    axs[row, col].set_title(f'block{i+1}_layer')
    axs[row, col].axis('off')

# Suppression des subplots vides
for i in range(len(grad_models) + 1, num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Tomato___Leaf_Mold

In [ ]:
import pickle
import tensorflow as tf
import cv2
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt

response = requests.get('https://drive.google.com/uc?export=download&id=1i8jQhllkUrHbRO9g5jI5sUl2uDyv7Rvz')
img1 = Image.open(BytesIO(response.content))

input_shape = (256, 256, 3)

# Chargement du modèle
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Modèles de gradient pour chaque couche
grad_models = [
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_pool').output, model.output])
]

# Prétraitement de l'image
img = img1.resize((256, 256))
img = np.array(img) / 255.0
img = np.expand_dims(img, axis=0)

num_rows = 5
num_cols = 4
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))

# Image originale
axs[0, 0].imshow(img1)
axs[0, 0].set_title('Image originale')
axs[0, 0].axis('off')

for i, grad_model in enumerate(grad_models):
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img)
        label_idx = tf.argmax(tf.squeeze(predictions)).numpy()  # Index de la classe prédite
        loss = tf.gather(tf.squeeze(predictions), [label_idx])

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]
    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = gate_f * gate_r * grads
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, output), axis=-1)
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (256, 256))
    heatmap = cam / np.max(cam)
    overlay = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    gradcam = cv2.addWeighted(np.array(img1), 0.5, overlay, 0.5, 0)

    row = (i + 1) // num_cols
    col = (i + 1) % num_cols

    axs[row, col].imshow(gradcam, cmap='jet')
    axs[row, col].set_title(f'block{i+1}_layer')
    axs[row, col].axis('off')

# Suppression des subplots vides
for i in range(len(grad_models) + 1, num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [15]:
import pickle
import tensorflow as tf
import cv2
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
import gdown

response = requests.get('https://drive.google.com/uc?export=download&id=1i8jQhllkUrHbRO9g5jI5sUl2uDyv7Rvz')
img1 = Image.open(BytesIO(response.content))

input_shape = (256, 256, 3)

# Chargement du modèle
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Modèles de gradient pour chaque couche
grad_models = [
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block1_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block2_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block3_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block4_pool').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv1').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv2').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_conv3').output, model.output]),
    tf.keras.models.Model([model.input], [model.get_layer('block5_pool').output, model.output])
]

# Prétraitement de l'image
img = img1.resize((256, 256))
img = np.array(img) / 255.0
img = np.expand_dims(img, axis=0)

num_rows = 5
num_cols = 4
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))

# Image originale
axs[0, 0].imshow(img1)
axs[0, 0].set_title('Image originale')
axs[0, 0].axis('off')

for i, grad_model in enumerate(grad_models):
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img)
        label_idx = tf.argmax(tf.squeeze(predictions)).numpy()  # Index de la classe prédite
        loss = tf.gather(tf.squeeze(predictions), [label_idx])

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]
    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = gate_f * gate_r * grads
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, output), axis=-1)
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (256, 256))
    heatmap = cam / np.max(cam)
    overlay = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    gradcam = cv2.addWeighted(np.array(img1), 0.5, overlay, 0.5, 0)

    row = (i + 1) // num_cols
    col = (i + 1) % num_cols

    axs[row, col].imshow(gradcam, cmap='jet')
    axs[row, col].set_title(f'block{i+1}_layer')
    axs[row, col].axis('off')

# Suppression des subplots vides
for i in range(len(grad_models) + 1, num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()

def grad_cam(input_model, image, cls, layer_name):
    """Méthode GradCAM pour visualiser la saillance de l'entrée."""
    y_c = input_model.output[0, cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = tf.gradients(y_c, conv_output)[0]
    # Normalisation si nécessaire
    # grads = normalize(grads)
    gradient_function = tf.function([input_model.input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # Traitement du CAM
    cam = cv2.resize(cam, (W, H), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam

def build_model():
    # Construction du modèle principal
    input_shape = (256, 256, 3)
    model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Ajout des couches supplémentaires
    x = model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    # Construction du modèle final avec les couches ajoutées
    model = tf.keras.models.Model(inputs=model.input, outputs=x)

    return model

    return model

def build_guided_model():
    # Construction du modèle guidé
    guided_model = tf.keras.models.clone_model(build_model())
    guided_model.set_weights(build_model().get_weights())

    return guided_model

def grad_cam_batch(input_model, images, classes, layer_name):
    """Méthode GradCAM pour visualiser la saillance de l'entrée.
    Identique à grad_cam mais traite plusieurs images en une seule exécution."""
    loss = tf.gather_nd(input_model.output, np.dstack([range(images.shape[0]), classes])[0])
    layer_output = input_model.get_layer(layer_name).output
    grads = tf.gradients(loss, layer_output)[0]
    gradient_fn = tf.function([input_model.input, tf.keras.backend.learning_phase()], [layer_output, grads])

    conv_output, grads_val = gradient_fn([images, 0])
    weights = np.mean(grads_val, axis=(1, 2))
    cams = np.einsum('ijkl,il->ijk', conv_output, weights)

    # Traitement des CAMs
    new_cams = np.empty((images.shape[0], H, W))
    for i in range(new_cams.shape[0]):
        cam_i = cams[i] - cams[i].mean()
        cam_i = (cam_i + 1e-10) / (np.linalg.norm(cam_i, 2) + 1e-10)
        new_cams[i] = cv2.resize(cam_i, (W, H), cv2.INTER_LINEAR)
        new_cams[i] = np.maximum(new_cams[i], 0)
        new_cams[i] = new_cams[i] / new_cams[i].max()

    return new_cams

def compute_saliency(model, guided_model, img_path, layer_name='block5_conv3', cls=-1, visualize=True, save=True):
    """Calcule la saillance en utilisant les trois approches.
        -layer_name : couche pour calculer les gradients ;
        -cls : numéro de classe à localiser (-1 pour la classe la plus probable).
    """

    url = 'https://drive.google.com/uc?export=download&id=1i8jQhllkUrHbRO9g5jI5sUl2uDyv7Rvz'
    output = 'plante.JPG'


# Ouvrir l'image avec PIL
    image = Image.open(output)
    gradcam, gb, guided_gradcam = compute_saliency(model, guided_model, img, layer_name='block5_conv3',
                                               cls=-1, visualize=True, save=True)
    preprocessed_input = img1                          #load_image(img_path)

    predictions = model.predict(preprocessed_input)
    top_n = 5
    top = decode_predictions(predictions, top=top_n)[0]
    classes = np.argsort(predictions[0])[-top_n:][::-1]
    print('Prédiction du modèle :')
    for c, p in zip(classes, top):
        print('\t{:15s}\t({})\tavec une probabilité de {:.3f}'.format(p[1], c, p[2]))
    if cls == -1:
        cls = np.argmax(predictions)
    class_name = decode_predictions(np.eye(1, 1000, cls))[0][0][1]
    print("Explication pour '{}'".format(class_name))

    gradcam = grad_cam(model, preprocessed_input, cls, layer_name)
    gb = guided_backprop(guided_model, preprocessed_input, layer_name)
    guided_gradcam = gb * gradcam[..., np.newaxis]

    if save:
        jetcam = cv2.applyColorMap(np.uint8(255 * gradcam), cv2.COLORMAP_JET)
        jetcam = np.float32(jetcam) + load_image(img1, preprocess=False) / 2
        cv2.imwrite('gradcam.jpg', np.uint8(jetcam))
        cv2.imwrite('guided_backprop.jpg', deprocess_image(gb[0]))
        cv2.imwrite('guided_gradcam.jpg', deprocess_image(guided_gradcam[0]))

    if visualize:
        plt.figure(figsize=(15, 10))
        plt.subplot(131)
        plt.title('GradCAM')
        plt.axis('off')
        plt.imshow(load_image(img_path, preprocess=False))
        plt.imshow(gradcam, cmap='jet', alpha=0.5)

        plt.subplot(132)
        plt.title('Guided Backprop')
        plt.axis('off')
        plt.imshow(np.flip(deprocess_image(gb[0]), -1))

        plt.subplot(133)
        plt.title('Guided GradCAM')
        plt.axis('off')
        plt.imshow(np.flip(deprocess_image(guided_gradcam[0]), -1))
        plt.show()

    return gradcam, gb, guided_gradcam

model = build_model()
guided_model = build_guided_model()

gradcam, gb, guided_gradcam = compute_saliency(model, guided_model, 'cat_dog.png', layer_name='block5_conv3',
                                               cls=-1, visualize=True, save=True)
gradcam, gb, guided_gradcam = compute_saliency(model, guided_model, 'cat_dog.png', layer_name='block5_conv3',
                                               cls=282, visualize=True, save=False)

UnidentifiedImageError: ignored